In [2]:
#INIZIALIZZAIZIONE SESSION E AVVIO SPARK
import pyspark as pys 
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import pyspark.sql as sql

#Spark session
spark = SparkSession.builder.getOrCreate()

#Spark context
sc = spark.sparkContext


21/07/14 20:59:06 WARN Utils: Your hostname, lorenzo-X555UJ resolves to a loopback address: 127.0.1.1; using 192.168.43.245 instead (on interface wlp3s0)
21/07/14 20:59:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/07/14 20:59:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
#CARICAMENTO DF

forest_path = './learningPySpark/Data/forest_coverage_type.csv'

forest=spark.read.csv(
    forest_path,
    header=True,
    inferSchema=True
)

forest.printSchema()
#Sono tutti campi numerici

root
 |-- Elevation: integer (nullable = true)
 |-- Aspect: integer (nullable = true)
 |-- Slope: integer (nullable = true)
 |-- Horizontal_Distance_To_Hydrology: integer (nullable = true)
 |-- Vertical_Distance_To_Hydrology: integer (nullable = true)
 |-- Horizontal_Distance_To_Roadways: integer (nullable = true)
 |-- Hillshade_9am: integer (nullable = true)
 |-- Hillshade_Noon: integer (nullable = true)
 |-- Hillshade_3pm: integer (nullable = true)
 |-- Horizontal_Distance_To_Fire_Points: integer (nullable = true)
 |-- Wilderness_Area_Rawah: integer (nullable = true)
 |-- Wilderness_Area_Neota: integer (nullable = true)
 |-- Wilderness_Area_Comanche: integer (nullable = true)
 |-- Wilderness_Area_CacheLaPoudre: integer (nullable = true)
 |-- Soil_type_2702: integer (nullable = true)
 |-- Soil_type_2703: integer (nullable = true)
 |-- Soil_type_2704: integer (nullable = true)
 |-- Soil_type_2705: integer (nullable = true)
 |-- Soil_type_2706: integer (nullable = true)
 |-- Soil_type_2

In [4]:
#Transformers
import pyspark.sql.functions as f
import pyspark.ml.feature as feat
import numpy as np

#Bucketizer
bucket_no=10 

dist_min_max = (
    forest.agg( #prendo min e max
        f.min('Horizontal_Distance_To_Hydrology').alias("min")
        ,f.max('Horizontal_Distance_To_Hydrology').alias("max")
    ).rdd
    .map(lambda row: (row.min,row.max))
    .collect()[0] #Non prenso la lista ma solo il primo (unico) elemento

)
#range
rng = dist_min_max[1]-dist_min_max[0]

#Prendo 11 punti equidistanti nel range
#Valori limite di ogni bucket (11 tagli = 10 bucket)
#Nei tagli contano anche gli estrmi
splits = list(np.arange(
    dist_min_max[0],
    dist_min_max[1],
    rng/ (bucket_no+1)
)
)

#Creazione oggetto
bucketizer = feat.Bucketizer(
    splits=splits, #array con valori in cui tagliare
    inputCol="Horizontal_Distance_To_Hydrology",
    outputCol="Horizontal_Distance_To_Hydrology_Bkt"
)

#Trasformazione
(
    bucketizer.transform(forest).select(
        'Horizontal_Distance_To_Hydrology'
        ,'Horizontal_Distance_To_Hydrology_Bkt'
    ).show(5)
)

#DF con 2 colonne, il valori continuo e il numero di bucket

+--------------------------------+------------------------------------+
|Horizontal_Distance_To_Hydrology|Horizontal_Distance_To_Hydrology_Bkt|
+--------------------------------+------------------------------------+
|                             258|                                 2.0|
|                             212|                                 1.0|
|                             268|                                 2.0|
|                             242|                                 1.0|
|                             153|                                 1.0|
+--------------------------------+------------------------------------+
only showing top 5 rows



In [5]:
#Gli estimators (vedi sotto) vogliono una sola colonna
#Effettuiamo in raggrupamento

#raggruppiamo tutte e colonne nella colonna feat
vectorAssembler = (
    feat.VectorAssembler(inputCols=forest.columns,outputCol='feat')
)

#Prendiamo le 5 variabili (feature) più significative
pca = (
    feat.PCA(k=5,inputCol=vectorAssembler.getOutputCol(),outputCol='pca_feat')
)

(
    pca
    .fit(vectorAssembler.transform(forest))
    .transform(vectorAssembler.transform(forest))
    .select("feat","pca_feat")
    .take(1)
)

21/07/14 21:00:48 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
21/07/14 21:00:48 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK
21/07/14 21:00:49 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


[Row(feat=SparseVector(55, {0: 2596.0, 1: 51.0, 2: 3.0, 3: 258.0, 5: 510.0, 6: 221.0, 7: 232.0, 8: 148.0, 9: 6279.0, 10: 1.0, 42: 1.0, 54: 5.0}), pca_feat=DenseVector([-3887.7711, 4996.8103, 2323.0932, 1014.5873, -135.1702]))]

In [8]:
#ESTIMATORS
import pyspark.ml.classification as cl

#Cerchiamo di predirre quando la foresta avrà "cover type" = 1 (spruce-fir conifere)

#Creiamo un SVM MODEL
vectorAssembler = feat.VectorAssembler(
    inputCols=forest.columns[0:-1] #selezioniamo tutte le colonne tranne l'ultima (cover type)
    , outputCol="features"
)

fir_dataset = (
    vectorAssembler.transform(forest).withColumn( #aggiunge colonne
        # (NomeColonna,Dati)
        "label"
        ,(f.col("CoverType")==1).cast("integer") #1 = conifera
    )
    .select("label","features")
)

svc_obj = cl.LinearSVC(maxIter=10,regParam=0.01)
svc_model = svc_obj.fit(fir_dataset)

21/07/14 21:12:36 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
21/07/14 21:12:36 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [9]:
#Estraiamo il risultato della modellizazione
svc_model.coefficients

DenseVector([-0.0001, -0.0, -0.0023, -0.0, -0.0001, 0.0, -0.001, -0.0017, -0.0003, -0.0, 0.0, 0.0401, -0.0071, -0.0958, -0.0901, -0.0653, -0.0655, -0.0437, -0.0928, -0.0848, -0.0211, -0.0045, -0.0498, -0.0829, -0.0522, -0.0325, -0.0263, -0.0923, -0.0889, -0.0275, -0.0606, -0.0595, 0.0341, -0.003, 0.0822, 0.0607, 0.0351, 0.0093, 0.0048, -0.0154, 0.0422, -0.0673, -0.0039, -0.0142, 0.0036, 0.0078, 0.0, -0.0117, 0.0283, -0.0002, -0.0463, 0.0394, 0.0292, 0.0358])

In [13]:
#Regressione lineare
import pyspark.ml.regression as rg

#Cerchiamo di stimare l'altezza di una foresta

vectorAssembler = feat.VectorAssembler(
    inputCols=forest.columns[1:]
    ,outputCol="features"
)

elevation_dataset = (
    vectorAssembler.transform(forest).withColumn( #aggiunge colonne
        "label",
        f.col("Elevation").cast("float")#regressione con variabili continue
    )
    .select("label","features")
)

#Modello regressione lineare
lr_obj = rg.LinearRegression(maxIter=10,regParam=0.01,elasticNetParam=1.00)
lr_model=lr_obj.fit(elevation_dataset)

#Il codice per creare un modello è abbastanza standard è di facile realizzazione

In [20]:
#Risultati della modellazione
lr_model.coefficients

#Resoconto automatico
summary = lr_model.summary

print(
    summary.r2 #78% buon risultato
    , summary.rootMeanSquaredError
    ,summary.meanAbsoluteError
)

0.7860412464754158 129.50871925702575 103.34079732698777


0.7860412464754158 129.50871925702575 103.34079732698777
